#Задача
Сравнить несколько подходов к получению векторных представлений текста

# Детали проведения экспериментов
Я решила сравнить 2 подхода к построению эмбеддингов для предложений: агрегированные эмбеддинги Word2Vec и векторы, полученные с помощью SentenceTransformers. В качестве метрики я выбрала semantic textual similarity(STS), потому что она хорошо интерпретируема, для ее оценки есть множество широко используемых датасетов  и для проведения экспериментов с ней в самом простом варианте не нужно дополнительно дообучать какие-либо модели.

#Гипотеза:
Эмбеддинги, полученные с помощью SentenceTransformers, будут показывать более лучшие результаты в задаче semantic textual similarity, чем эмбеддинги для предложений, построенные с помощью агрегирования предобученных векторов Word2Vec для слов предложения.

Почему возникла эта гипотеза?

1) В этой статье https://arxiv.org/pdf/1908.10084.pdf были проведены похожие эксперименты с использованием average GloVe embeddings и SentenceTransformers embeddings. Видим, что второй подход дает результаты значительно лучше.

2) Агрегированные вектора по своей сути похожи на "мешок слов", то есть мы можем понять примерную семантику предложения по отдельным словам, но истинная суть текста очень сильно зависит от порядка слов в предложениях. Агрегированные вектора никак не могут учесть порядок слов предложений, а трансформерные могут, то есть и смысл текста трансформерные векторы будут передавать лучше агрегированных.


# Технические детали
1)Для оценки качества эмбеддингов я использую SentEval(https://github.com/facebookresearch/SentEval). Я провожу эксперименты на всех доступных semantic textual similarity датасетах. В качестве основной метрики я использую коэффициент корреляции Спирмена между cosine-similarity эмбеддингов и лейблом в датасетах, потому что в статье про SentenceTransformers делается отсылка на другую статью авторов, в которой показано, что коэффициент корреляции Спирмена для оценки качества эмбеддингов предложений подходит лучше в задачах STS, чем коэффициент корреляции Пирсона.

2) Для получения агрегированных эмбеддингов я использую готовые предобученные вектора 'word2vec-google-news-300', которые я загружаю с помощью gensim-api(https://radimrehurek.com/gensim/apiref.html). Эмбеддинги для предложения получаются следующим образом: я привожу слова в нижний регистр, после разделяю предложение на токены, инициализирую результирующий вектор нулевым, нахожу вектор для каждого отдельного токена, если слова нет в датасете эмбеддингов, то пропускаю его, иначе прибавляю найденный вектор к результату.

3) Для трансформерных эмбеддингов я взяла готовую модель отсюда https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2. Я сразу применяю ее для получения эмбеддингов предложений без какого-либо дополнительного дообучения.

# Эксперименты

## Устанавливаем библиотеку SentEval и выбираем нужные задачи

In [ ]:
!rm -rf ./SentEval
!git clone https://github.com/facebookresearch/SentEval.git
%cd /content/SentEval/data/downstream

In [ ]:
!bash get_transfer_data.bash
%cd ..
%cd ..
!ls

In [110]:
import senteval
import numpy as np

PATH_TO_DATA = './data'
params_senteval = {'task_path': PATH_TO_DATA,
                   'usepytorch': True,
                   'kfold': 10}
tasks = [
    'STSBenchmark',
    'STS12',
    'STS13',
    'STS14',
    'STS15',
    'STS16',
]

## Агрегированные векторы Word2Vec

In [88]:
import gensim.downloader as api

model_w2v = api.load('word2vec-google-news-300')

In [111]:
def batcher_w2v(params, batch):
    batch = [sent if sent != [] else ['.'] for sent in batch]
    embeddings = []

    for sent in batch:
        sentvec = []
        for word in sent:
            if word in model_w2v:
                sentvec.append(model_w2v[word])
        if not sentvec:
            vec = np.zeros(300)
            sentvec.append(vec)
        sentvec = np.mean(sentvec, 0)
        embeddings.append(sentvec)

    return np.vstack(embeddings)

se = senteval.engine.SE(params_senteval, batcher_w2v)
results_w2v = se.eval(tasks)

results_w2v

[NeMo W 2023-10-02 02:56:53 nemo_logging:349] /content/SentEval/senteval/sts.py:42: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
      sent1 = np.array([s.split() for s in sent1])[not_empty_idx]
    
[NeMo W 2023-10-02 02:56:53 nemo_logging:349] /content/SentEval/senteval/sts.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
      sent2 = np.array([s.split() for s in sent2])[not_empty_idx]
    
[NeMo W 2023-10-02 02:56:54 nemo_logging:349] /content/SentEval/senteval/utils.py:39: RuntimeWarning: invalid value encountered in double_scalars
      return

{'STSBenchmark': {'devpearson': 0.7131674180528703,
  'pearson': 0.6423486032945015,
  'spearman': 0.6196792730335671,
  'mse': 1.6468258431225555,
  'yhat': array([1.70262126, 1.53167721, 2.08265509, ..., 3.70078188, 3.62982191,
         3.32651225]),
  'ndev': 1500,
  'ntest': 1379},
 'STS12': {'MSRpar': {'pearson': PearsonRResult(statistic=0.39744133044415353, pvalue=8.65068272332563e-30),
   'spearman': SignificanceResult(statistic=0.4090561420732993, pvalue=1.2814345402211917e-31),
   'nsamples': 750},
  'MSRvid': {'pearson': PearsonRResult(statistic=0.7810864972457823, pvalue=3.889587845740084e-155),
   'spearman': SignificanceResult(statistic=0.7646224554203609, pvalue=7.39402451804042e-145),
   'nsamples': 750},
  'SMTeuroparl': {'pearson': PearsonRResult(statistic=0.16272917720970084, pvalue=0.00046489553722674866),
   'spearman': SignificanceResult(statistic=0.4985593690637984, pvalue=3.2574023419063176e-30),
   'nsamples': 459},
  'surprise.OnWN': {'pearson': PearsonRResult(

##Sentence Transformers

In [ ]:
!pip install sentence-transformers

In [91]:
from sentence_transformers import SentenceTransformer

model_st = SentenceTransformer('all-MiniLM-L6-v2')

In [112]:
def batcher_st(params, batch):
    batch = [' '.join(sent) if sent else '.' for sent in batch]
    return model_st.encode(batch)

se = senteval.engine.SE(params_senteval, batcher_st)
results_st = se.eval(transfer_tasks)

results_st

[NeMo W 2023-10-02 02:57:35 nemo_logging:349] /content/SentEval/senteval/sts.py:42: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
      sent1 = np.array([s.split() for s in sent1])[not_empty_idx]
    
[NeMo W 2023-10-02 02:57:35 nemo_logging:349] /content/SentEval/senteval/sts.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
      sent2 = np.array([s.split() for s in sent2])[not_empty_idx]
    


{'STSBenchmark': {'devpearson': 0.8575836165538598,
  'pearson': 0.8209868406631583,
  'spearman': 0.8244550119836955,
  'mse': 1.0223146249185906,
  'yhat': array([1.82950827, 2.18019088, 1.74298608, ..., 4.0255325 , 4.36806493,
         2.73509503]),
  'ndev': 1500,
  'ntest': 1379},
 'STS12': {'MSRpar': {'pearson': PearsonRResult(statistic=0.5838885716612634, pvalue=9.523977883104918e-70),
   'spearman': SignificanceResult(statistic=0.578373326825346, pvalue=3.58948887280327e-68),
   'nsamples': 750},
  'MSRvid': {'pearson': PearsonRResult(statistic=0.931813139179239, pvalue=0.0),
   'spearman': SignificanceResult(statistic=0.9360861990092416, pvalue=0.0),
   'nsamples': 750},
  'SMTeuroparl': {'pearson': PearsonRResult(statistic=0.5754948058128788, pvalue=7.777842945063242e-42),
   'spearman': SignificanceResult(statistic=0.6368355034607547, pvalue=1.4129127890816277e-53),
   'nsamples': 459},
  'surprise.OnWN': {'pearson': PearsonRResult(statistic=0.7496061440869902, pvalue=3.4235

#Результаты

In [123]:
def collect_spearman(results):
  spearman = []
  for r in results:
    if r == 'STSBenchmark':
      spearman.append(results[r]['spearman'])
    else:
      spearman.append(results[r]['all']['spearman']['wmean'])
  return spearman

In [126]:
w2v = collect_spearman(results_w2v)
w2v

[0.6196792730335671,
 0.5749613157848575,
 0.633022637577695,
 0.6561185731432768,
 0.7039261762062249,
 0.660410884944213]

In [127]:
st = collect_spearman(results_st)
st

[0.8244550119836955,
 0.7106201570481677,
 0.7896985570155666,
 0.7721616242202547,
 0.838422998588126,
 0.8121285713696077]

In [131]:
import pandas as pd

data = {
    'Word2Vec_Avg' : w2v,
    'SentenceTransformers' : st
}

df = pd.DataFrame(data, index=tasks)
df

,Word2Vec_Avg,SentenceTransformers
STSBenchmark,0.619679,0.824455
STS12,0.574961,0.710620
STS13,0.633023,0.789699
STS14,0.656119,0.772162
STS15,0.703926,0.838423
STS16,0.660411,0.812129


## Вывод
Видим, что мы получили такие результаты, которые и ожидали (коэффициенты корреляции Спирмена у SentenceTransformers больше, поэтому можно предположить, что такой подход к построению эмбеддингов должен помогать лучше улавливать семантику текстов, чем агрегированные векторы Word2Vec)

## Ссылки на источники

https://arxiv.org/abs/1301.3781

https://arxiv.org/pdf/1908.10084

https://arxiv.org/abs/1803.05449

https://aclanthology.org/C16-1009/